<a href="https://colab.research.google.com/github/Lehula1/divergGit/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install plotly
! pip install numpy
! pip install yfinance
! pip install pandas_ta
! pip install matplotlib
#! pip install ta
! pip install beautifulsoup4
! pip install requests
! pip install datetime

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
from pandas_datareader import data as pdr
import pandas_ta as ta
import yfinance as yf
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from collections import deque

from plotly.subplots import make_subplots


import bs4 as bs
import requests
import datetime

from bs4 import BeautifulSoup

In [48]:
# Get SP500 tickers -------------------------------------------
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

ticker_list = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    ticker_list.append(ticker)

ticker_list = [s.replace('\n', '') for s in ticker_list]



all_symbols  = " ".join(ticker_list)

tickers = yf.Tickers(all_symbols)


#Cycle through stock tickers--------------------------------------------
for ticker in ticker_list:
    if (ticker != 'BRK.B') and (ticker != 'BF.B'):

      yfObj = yf.Ticker(ticker)
      data = yfObj.history(period='3mo', interval='1d')
      #data = yfObj.history("SPY", start="2023-09-01", end="2024-01-",  interval = "1d")
      exchange = yfObj.info['exchange']
      if exchange == "NYQ":
        exchange = "NYSE"
      elif exchange =='NMS':
        exchange = 'NASDAQ'
      close = data['High'].values
      dates = data.index

      colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

      maxPrice_index = argrelextrema(data['High'].values, np.greater, order=2)[0]
      minPrice_index = argrelextrema(data['Low'].values, np.less, order=2)[0]

      PriceH = 0
      PriceHH = 0

      # Get K consecutive higher peaks------------------------------------------
      K = 2
      highs = data.iloc[maxPrice_index]['High']

      data.ta.stoch(high='high', low='low', k=14, d=3, append=True)
      data.columns = [x.lower() for x in data.columns]
      max_stoch = argrelextrema(data['stochk_14_3_3'].values, np.greater, order=2)[0]
      stoch_highs = data.iloc[max_stoch]['stochk_14_3_3']

      #Get divergence on Stochastics chart-------------------------------------------------------------
      extremaStoch = []
      count1 = 0
      stoch_deque = deque(maxlen=K)
      for j, sto in enumerate(max_stoch):
        if j == 0:
          stoch_deque.append(sto)
          continue
        if stoch_highs[j] > stoch_highs[j-1] or data['high'].values[max_stoch[j-1]] > data['high'].values[max_stoch[j]]:

            stoch_deque.clear()

        stoch_deque.append(sto)

        if len(stoch_deque) == K:
          # K-consecutive higher highs found
          extremaStoch.append(stoch_deque.copy())

          count1 += 1
    if count1 >= 1 and data.index[extremaStoch[-1][0]] > data.index[50]:

      print(f"{exchange}:{ticker}" , end=",")




#Get price divergence--------------------------------------------------------
extrema = []
count = 0
ex_deque = deque(maxlen=K)
for i, idx in enumerate(maxPrice_index):
  if i == 0:
    ex_deque.append(idx)
    continue
  if highs[i] < highs[i-1] or data.iloc[maxPrice_index[i]]['stochk_14_3_3'] > data.iloc[maxPrice_index[i-1]]['stochk_14_3_3']:

    ex_deque.clear()

  ex_deque.append(idx)

  if len(ex_deque) == K:
    # K-consecutive higher highs found
    extrema.append(ex_deque.copy())

    count += 1




"""
#print(highs[i])
    #extrema count++++++++++++++++++++++++++++++++++++++++++++++++
    #print(extrema[count][0])
    #print(extrema[count][1])
    #print(data.iloc[maxPrice_index[i-1]]['stochk_14_3_3'])
    #print(highs[i])
    #print(data['stochk_14_3_3'].values['High'[i]])

    #data['high'].values[max_stoch[j-1]]

#print(highs)
print(data.iloc[max_idx])
print(ex_deque)
print(test)
print(extrema.index)
print(extrema[0][1])
print(extrema[1])
print(extrema[-1][1])
print(extrema)
print(max_idx)
print(data.index[max_idx])
print(data.iloc[max_idx,1])

candlestickHigh = data['high'].values[max_stoch[1]]
test = argrelextrema(data['high'].values, np.greater, order=2)[0]
print(candlestickHigh)
print(close)
print(extremaStoch)
print(stoch_highs)
print(data.iloc[extremaStoch[1]])

print(data['high'].values)
print(highs)
print(max_idx)

print(data['stochk_14_3_3'].values) #----------------------------------------------------------------------
print(data.iloc[max_stoch]['stochk_14_3_3'])
"""
ticker=""
# Create our primary chart
# the rows/cols arguments tell plotly we want two figures

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[1,.5], vertical_spacing = 0.01)

# Create our Candlestick chart with an overlaid price line
fig.append_trace(
    go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        showlegend=False
    ), row=1, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[maxPrice_index],
        y=data.iloc[maxPrice_index,1]+8.5,
        marker=dict(
            symbol='cross',
            color='green',
            size=10)))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[minPrice_index],
        y=data.iloc[minPrice_index,2]-5.5,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))

# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=data.index,
        y=data['stochk_14_3_3'],
        line=dict(color='#ff9900', width=2),
        name='fast'), row=2, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_stoch],
        y=data.iloc[max_stoch]['stochk_14_3_3'],
        marker=dict(
            symbol='cross',
            color='red',
            size=10)), row=2, col=1)

fig.add_trace(
    go.Scatter(
        x=[data.index[extremaStoch[-1][0]], data.index[extremaStoch[-1][1]]],
        y=[data['stochk_14_3_3'][extremaStoch[-1][0]], data['stochk_14_3_3'][extremaStoch[-1][1]]],
        mode='lines',
        name='trend line',
        line=dict(color='red')),row=2, col=1)

# Extend our y-axis a bit
fig.update_yaxes(range=[-10, 110], row=2, col=1)

# Add upper/lower bounds
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=20, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=80, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(values=["2023-11-23","2023-12-25", "2024-01-15","2024-01-01"])
      ]  # hide Christmas and New Year's
)

fig.update_layout(xaxis_rangeslider_visible=False,  width=1500, title=ticker,
     margin=dict(
        l=50,
        r=50,
        b=10,
        t=50,
        pad=4))


NYSE:ALL,NASDAQ:AMZN,NYSE:ANET,NYSE:AIZ,NYSE:CTLT,NASDAQ:CDW,NYSE:CVX,NASDAQ:CTSH,NYSE:COP,NASDAQ:CEG,NASDAQ:COST,NASDAQ:CSX,NYSE:CMI,NYSE:EG,NYSE:FI,NYSE:GPN,NYSE:HLT,NYSE:HRL,NYSE:IEX,NYSE:IR,NYSE:MPC,NASDAQ:MAR,NYSE:MA,NYSE:MRK,NASDAQ:META,NYSE:NRG,NYSE:OTIS,NYSE:PNR,NYSE:PHM,NYSE:RSG,NASDAQ:ROST,NYSE:CRM,NYSE:SNA,NYSE:TRGP,NYSE:UBER,NASDAQ:VRSK,NYSE:WMT,NYSE:WAT,NYSE:GWW,

In [50]:

#Cycle through stock tickers--------------------------------------------
ticker="CVX"
yfObj = yf.Ticker(ticker)
data = yfObj.history(period='3mo', interval='1d')
#data = yfObj.history("SPY", start="2023-09-01", end="2024-01-",  interval = "1d")
close = data['High'].values
dates = data.index

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

maxPrice_index = argrelextrema(data['High'].values, np.greater, order=2)[0]
minPrice_index = argrelextrema(data['Low'].values, np.less, order=2)[0]

PriceH = 0
PriceHH = 0

# Get K consecutive higher peaks------------------------------------------
K = 2
highs = data.iloc[maxPrice_index]['High']

data.ta.stoch(high='high', low='low', k=14, d=3, append=True)
data.columns = [x.lower() for x in data.columns]
max_stoch = argrelextrema(data['stochk_14_3_3'].values, np.greater, order=2)[0]
stoch_highs = data.iloc[max_stoch]['stochk_14_3_3']

#Get divergence on Stochastics chart-------------------------------------------------------------
extremaStoch = []
count1 = 0
stoch_deque = deque(maxlen=K)
for j, sto in enumerate(max_stoch):
  if j == 0:
    stoch_deque.append(sto)
    continue
  if stoch_highs[j] > stoch_highs[j-1] or data['high'].values[max_stoch[j-1]] > data['high'].values[max_stoch[j]]:

      stoch_deque.clear()

  stoch_deque.append(sto)

  if len(stoch_deque) == K:
    # K-consecutive higher highs found
    extremaStoch.append(stoch_deque.copy())

    count1 += 1





#Get price divergence--------------------------------------------------------
extrema = []
count = 0
ex_deque = deque(maxlen=K)
for i, idx in enumerate(maxPrice_index):
  if i == 0:
    ex_deque.append(idx)
    continue
  if highs[i] < highs[i-1] or data.iloc[maxPrice_index[i]]['stochk_14_3_3'] > data.iloc[maxPrice_index[i-1]]['stochk_14_3_3']:

    ex_deque.clear()

  ex_deque.append(idx)

  if len(ex_deque) == K:
    # K-consecutive higher highs found
    extrema.append(ex_deque.copy())

    count += 1




"""
#print(highs[i])
    #extrema count++++++++++++++++++++++++++++++++++++++++++++++++
    #print(extrema[count][0])
    #print(extrema[count][1])
    #print(data.iloc[maxPrice_index[i-1]]['stochk_14_3_3'])
    #print(highs[i])
    #print(data['stochk_14_3_3'].values['High'[i]])

    #data['high'].values[max_stoch[j-1]]

#print(highs)
print(data.iloc[max_idx])
print(ex_deque)
print(test)
print(extrema.index)
print(extrema[0][1])
print(extrema[1])
print(extrema[-1][1])
print(extrema)
print(max_idx)
print(data.index[max_idx])
print(data.iloc[max_idx,1])

candlestickHigh = data['high'].values[max_stoch[1]]
test = argrelextrema(data['high'].values, np.greater, order=2)[0]
print(candlestickHigh)
print(close)
print(extremaStoch)
print(stoch_highs)
print(data.iloc[extremaStoch[1]])

print(data['high'].values)
print(highs)
print(max_idx)

print(data['stochk_14_3_3'].values) #----------------------------------------------------------------------
print(data.iloc[max_stoch]['stochk_14_3_3'])
"""

# Create our primary chart
# the rows/cols arguments tell plotly we want two figures

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[1,.5], vertical_spacing = 0.01)

# Create our Candlestick chart with an overlaid price line
fig.append_trace(
    go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        showlegend=False
    ), row=1, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[maxPrice_index],
        y=data.iloc[maxPrice_index,1]+8.5,
        marker=dict(
            symbol='cross',
            color='green',
            size=10)))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[minPrice_index],
        y=data.iloc[minPrice_index,2]-5.5,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))

# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=data.index,
        y=data['stochk_14_3_3'],
        line=dict(color='#ff9900', width=2),
        name='fast'), row=2, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_stoch],
        y=data.iloc[max_stoch]['stochk_14_3_3'],
        marker=dict(
            symbol='cross',
            color='red',
            size=10)), row=2, col=1)

fig.add_trace(
    go.Scatter(
        x=[data.index[extremaStoch[-1][0]], data.index[extremaStoch[-1][1]]],
        y=[data['stochk_14_3_3'][extremaStoch[-1][0]], data['stochk_14_3_3'][extremaStoch[-1][1]]],
        mode='lines',
        name='trend line',
        line=dict(color='red')),row=2, col=1)

# Extend our y-axis a bit
fig.update_yaxes(range=[-10, 110], row=2, col=1)

# Add upper/lower bounds
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=20, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=80, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(values=["2023-11-23","2023-12-25", "2024-01-15","2024-01-01"])
      ]  # hide Christmas and New Year's
)

fig.update_layout(xaxis_rangeslider_visible=False,  width=1500, title=ticker,
     margin=dict(
        l=50,
        r=50,
        b=10,
        t=50,
        pad=4))


In [ ]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
countries = stock_data.get_all_countries()
indices = stock_data.get_all_indices()
industries = stock_data.get_all_industries()

#You can select all stocks of an index as follows:

from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
german_stocks = stock_data.get_stocks_by_index('DAX')
uk_stocks = stock_data.get_stocks_by_index('FTSE 100')
us_stocks = stock_data.get_stocks_by_index('S&P 500')

#print(list(us_stocks))

#If you are only interested in ticker symbols, then you should have a look at the following lines:

from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
# the naming conversation is get_{index_name}_{exchange_city}_{yahoo or google}_tickers
dax_google = stock_data.get_dax_frankfurt_google_tickers()
dax_yahoo = stock_data.get_dax_frankfurt_yahoo_tickers()
sp100_yahoo = stock_data.get_sp_100_nyc_yahoo_tickers()
sp500_google = stock_data.get_sp_500_nyc_google_tickers()
dow_yahoo = stock_data.get_dow_jones_nyc_yahoo_tickers()
# there are too many combination. Here is a complete list of all getters
all_ticker_getter_names = list(filter(
   lambda x: (
         x.endswith('_google_tickers') or x.endswith('_yahoo_tickers')
   ),
   dir(stock_data),
))
print(all_ticker_getter_names)

['get_aex_frankfurt_google_tickers', 'get_aex_frankfurt_yahoo_tickers', 'get_aex_london_google_tickers', 'get_aex_london_yahoo_tickers', 'get_aex_moscow_google_tickers', 'get_aex_moscow_yahoo_tickers', 'get_aex_nyc_google_tickers', 'get_aex_nyc_yahoo_tickers', 'get_bel_20_frankfurt_google_tickers', 'get_bel_20_frankfurt_yahoo_tickers', 'get_bel_20_london_google_tickers', 'get_bel_20_london_yahoo_tickers', 'get_bel_20_moscow_google_tickers', 'get_bel_20_moscow_yahoo_tickers', 'get_bel_20_nyc_google_tickers', 'get_bel_20_nyc_yahoo_tickers', 'get_cac_40_frankfurt_google_tickers', 'get_cac_40_frankfurt_yahoo_tickers', 'get_cac_40_london_google_tickers', 'get_cac_40_london_yahoo_tickers', 'get_cac_40_moscow_google_tickers', 'get_cac_40_moscow_yahoo_tickers', 'get_cac_40_nyc_google_tickers', 'get_cac_40_nyc_yahoo_tickers', 'get_cac_mid_60_frankfurt_google_tickers', 'get_cac_mid_60_frankfurt_yahoo_tickers', 'get_cac_mid_60_london_google_tickers', 'get_cac_mid_60_london_yahoo_tickers', 'get_ca

In [35]:
ticker="EG"
yfObj = yf.Ticker(ticker)
data = yfObj.history(period='3mo', interval='1d')
#data = yfObj.history("SPY", start="2023-09-01", end="2024-01-",  interval = "1d")
print(yfObj.info['exchange'])


NYQ
